##### **Importing necessary libraries**

In [88]:
import pandas as pd 
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [90]:
def evaluation_metrics(y_test,y_pred):
    r2 = r2_score(y_test,y_pred)
    return r2

##### **Model Training**

In [89]:
import pandas as pd

df = pd.read_csv(r'F:\GUVI_DATA_SCIENCE\Project\Smart-Predictive-Modeling-for-Rental-Property-Prices\Dataset\analysis_df.csv')
df.head()

,type,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,rent
0,BHK2,12.929557,77.672280,ANYONE,0,1,0,1,SEMI_FURNISHED,BOTH,1400,4.0,2.0,NE,2.0,3.0,4.0,CORPORATION,AP,2.0,22000.0
1,BHK3,12.982870,80.262012,FAMILY,0,1,0,0,SEMI_FURNISHED,BOTH,1350,6.0,3.0,E,3.0,1.0,5.0,CORPORATION,AP,3.0,28000.0
2,BHK1,12.955991,77.531634,FAMILY,0,0,0,1,SEMI_FURNISHED,TWO_WHEELER,600,3.0,1.0,E,1.0,1.0,2.0,CORPORATION,IH,0.0,8000.0
3,BHK3,12.963903,77.649446,FAMILY,0,0,0,1,SEMI_FURNISHED,BOTH,1500,15.0,3.0,E,4.0,0.0,0.0,CORPORATION,IH,1.0,45000.0
4,BHK3,12.967144,77.750662,ANYONE,1,1,1,1,FULLY_FURNISHED,BOTH,1895,5.0,3.0,NE,5.0,3.0,9.0,CORPORATION,AP,3.0,32000.0


In [91]:
x = df.drop(['latitude','longitude','rent'], axis = 1)
y = df['rent']

print(x.shape)
print(y.shape)

(17012, 18)
(17012,)


In [92]:
x.describe()

,gym,lift,swimming_pool,negotiable,property_size,property_age,bathroom,cup_board,floor,total_floor,balconies
count,17012.000000,17012.000000,17012.000000,17012.000000,17012.000000,17012.000000,17012.000000,17012.000000,17012.000000,17012.000000,17012.000000
mean,0.199095,0.328004,0.147602,0.715730,1057.368857,6.889842,1.845815,2.214966,1.711028,3.469845,1.130966
std,0.399331,0.469500,0.354715,0.451079,613.524439,5.607976,0.713688,1.582874,1.955555,2.767769,0.997033
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,700.000000,3.000000,1.000000,1.000000,0.000000,2.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,1005.000000,5.000000,2.000000,2.000000,1.000000,3.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1270.000000,10.000000,2.000000,3.000000,2.000000,4.000000,2.000000
max,1.000000,1.000000,1.000000,1.000000,50000.000000,100.000000,21.000000,40.000000,25.000000,26.000000,13.000000


In [93]:
import numpy as np

x['property_size'] = np.log(x['property_size'])

In [94]:
x['property_age'] = np.log(x['property_age'])

In [95]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(13609, 18)
(3403, 18)
(13609,)
(3403,)


In [96]:
x_train.sample()

,type,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies
13093,BHK2,BACHELOR,0,0,0,1,SEMI_FURNISHED,BOTH,6.802395,1.791759,2.0,E,2.0,1.0,4.0,CORPORATION,IF,3.0


In [97]:
# Encoding Categorical Columns

transformer = ColumnTransformer([('trans1', OneHotEncoder(sparse_output = False, handle_unknown = 'error', drop = 'first'), [0,1,2,3,4,5,6,7,11,15,16])], remainder = 'passthrough')

x_train_transformed = transformer.fit_transform(x_train)
x_test_transformed  = transformer.transform(x_test)

In [81]:
# Hyperparamater Tuning

from sklearn.model_selection import GridSearchCV

# Defining a Base model
base_model = RandomForestRegressor(random_state = 42)

# Make a dictionary of hyperparameters values to search
search_space = {"n_estimators":[100,120], "bootstrap" : [True, False]}

# making an GridSearchCV Object
GS = GridSearchCV(estimator = base_model,
                  param_grid = search_space,
                  scoring = 'r2',
                  refit = 'r2', # this will return the model that is tested with the r2 metrices
                  cv = 3,
                  verbose = 4)

GS.fit(x_train_transformed,y_train)

best_params = GS.best_params_ # To get only the best hyperparameter values that we searched for
n_est = best_params.values()
print('n_estimator',n_est)

# Training data metrics

best_score = GS.best_score_ # score according to the metric we passes in refit
print('R2 Score :',best_score)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END ..bootstrap=True, n_estimators=100;, score=0.672 total time=  16.6s
[CV 2/3] END ..bootstrap=True, n_estimators=100;, score=0.667 total time=  15.4s
[CV 3/3] END ..bootstrap=True, n_estimators=100;, score=0.670 total time=  15.3s
[CV 1/3] END ..bootstrap=True, n_estimators=120;, score=0.673 total time=  18.3s
[CV 2/3] END ..bootstrap=True, n_estimators=120;, score=0.667 total time=  18.5s
[CV 3/3] END ..bootstrap=True, n_estimators=120;, score=0.669 total time=  18.3s
[CV 1/3] END .bootstrap=False, n_estimators=100;, score=0.408 total time=  22.0s
[CV 2/3] END .bootstrap=False, n_estimators=100;, score=0.418 total time=  21.1s
[CV 3/3] END .bootstrap=False, n_estimators=100;, score=0.393 total time=  20.8s
[CV 1/3] END .bootstrap=False, n_estimators=120;, score=0.409 total time=  25.1s
[CV 2/3] END .bootstrap=False, n_estimators=120;, score=0.418 total time=  25.0s
[CV 3/3] END .bootstrap=False, n_estimators=120;,

In [85]:
final_model = RandomForestRegressor(n_estimators = 100, bootstrap = True, random_state = 42)
final_model.fit(x_train_transformed, y_train)

RandomForestRegressor(random_state=42)

In [104]:
# Testing data metrics

y_pred = final_model.predict(x_test_transformed)

score = evaluation_metrics(y_test, y_pred)
print('The r2 score is',round(score, 1) * 100, '%')

The r2 score is 70.0 %


In [109]:
# Saving the model

import pickle

pickle.dump(final_model, open(r'F:\GUVI_DATA_SCIENCE\Project\Smart-Predictive-Modeling-for-Rental-Property-Prices\Model\reg_model.pkl', 'wb'))

In [111]:
# Saving the transformer

pickle.dump(transformer, open(r'F:\GUVI_DATA_SCIENCE\Project\Smart-Predictive-Modeling-for-Rental-Property-Prices\Artifacts\transformer.pkl','wb'))

In [122]:
# User input prediction

type = 'BHK2'
lease_type = 'BACHELOR'
gym	= 0
lift = 0	
swimming_pool = 0
negotiable = 1
furnishing = 'SEMI_FURNISHED'
parking = 'BOTH'
property_size = 1000
log_property_size = np.log(property_size)
property_age = 10
log_property_age = np.log(property_age)
bathroom = 2
facing = 'E'
cup_board = 2
floor = 1
total_floor = 4
water_supply = 'CORPORATION'
building_type = 'IF'
balconies = 3

user_df = pd.DataFrame([[type,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies]], columns = ['type', 'lease_type', 'gym', 'lift', 'swimming_pool', 'negotiable', 'furnishing', 'parking', 'property_size', 'property_age', 'bathroom', 'facing', 'cup_board', 'floor', 'total_floor', 'water_supply', 'building_type', 'balconies'])
user_df

tranformed_df = transformer.transform(user_df)

y_pred = final_model.predict(tranformed_df)
print('The rental Price is', round(y_pred[0]))

The rental Price is 21030
